# Experiments on PDF data extraction

The goal of this notebook is to run experiments to create an algorithm that can: 
1. Extract data from DanskeBank quaterly statements into a structured form
2. Compare that data with the user's expenses in Toto and create possible matches, for automated reconciliation (consolidation)

## 1. Extraction of data from the kontoudskrift PDF

The first step requires the extract of the data from the *Kontoudskrift* PDF statement received from DanskeBank.<br>
After experimenting, I've created a utility class `KudExtract` that can do this with the method `process_pdf`.<br>
That method will return a list of JSON objects (dicts) that contain a date, text (description) and amount (negative for expenses, positive for incomes).

In [45]:
import sys
import os
import pandas as pd

sys.path.append('../')

from src.KudExtract import KudExtract

In [2]:
def extract_data_from_kud(filepath, year, month, append_to_file=None): 

    # Starting from 2020.09 the kud switches to ',' as a decimal separator
    dec_separator = '.'
    thousands_separator = ','
    if int(year) > 2020: 
        dec_separator = ','
        thousands_separator = '.'
    elif int(year) == 2020 and int(month) >= 9: 
        dec_separator = ','
        thousands_separator = '.'

    kud = KudExtract(year, decimal_separator=dec_separator, thousands_separator=thousands_separator)

    kud_data = kud.process_pdf(filepath)

    if not append_to_file == None: 
        # Save it to a file
        with open(append_to_file, 'a') as file:
            for item in kud_data: 
                file.write(f"{item}\n")

    return kud_data

In [4]:
dir = "kuds/"
prefix = dir + "kud-"

kud_data_filename = 'kud_data.txt'

# Clear out previous files
if os.path.exists(kud_data_filename): 
    os.remove(kud_data_filename)

# Extract the data for every trimester
kud_files_list = os.listdir(dir)

for filename in kud_files_list: 
    
    filepath = dir + filename;
    
    year = filepath[len(prefix):len(prefix) + 4]
    month = filepath[-6:-4]
    
    kud_data = extract_data_from_kud(filepath, year, month, kud_data_filename)


Processing Kontoudskrift kuds/kud-2019.06.PDF for year 2019
Extracted 758 lines
Final lines retained: 136
Processing Kontoudskrift kuds/kud-2021.03.PDF for year 2021
Extracted 214 lines
Final lines retained: 66
Processing Kontoudskrift kuds/kud-2023.06.PDF for year 2023
Extracted 1081 lines
Final lines retained: 196
Processing Kontoudskrift kuds/kud-2019.03.PDF for year 2019
Extracted 522 lines
Final lines retained: 120
Processing Kontoudskrift kuds/kud-2021.12.PDF for year 2021
Extracted 1019 lines
Final lines retained: 136
Processing Kontoudskrift kuds/kud-2023.03.PDF for year 2023
Extracted 505 lines
Final lines retained: 120
Processing Kontoudskrift kuds/kud-2020.09.PDF for year 2020
Extracted 474 lines
Final lines retained: 104
Processing Kontoudskrift kuds/kud-2022.09.PDF for year 2022
Extracted 1046 lines
Final lines retained: 178
Processing Kontoudskrift kuds/kud-2018.09.PDF for year 2018
Extracted 388 lines
Final lines retained: 141
Processing Kontoudskrift kuds/kud-2020.03.PD

## 2. Compare and Predict

In this section we experiment on how to compare and predict similarity between the recorded Toto expenses and the ones extracted from the DanskeBank statement. 

To do that we have the following logical steps: 

1. Generate Embeddings from the expenses descriptions both in Toto and Kud 

2. Create possible matches based on the amount.<br>
Start by easy-matching expenses: 
    * When there's a 1:1 match between a Toto and a Kud expense, you have a match, no need to go further
    * When there's a 1:many match, you need an algo that will help you choose which one is the actual match
<br><br>

3. In the 1:many situation, use an algorithm to compare embeddings (e.g. Cosine Similarity). <br>
Concretely, that means comparing each KUD expense to each Toto Expense in the 1:many match and generating a similarity matrix. <br>
That gives a **similarity score** ($S_j^{(i)}$) to the j-eth match in the i-eth 1:many match.

4. In the 1:many situation, also do a similarity match between the dates.<br>
You can approximate that by converting dates to YYYYMMDD integers and calculating the sqared difference: <br>
$D_j^{(i)} = (D_k - D_t)^2$

5. Train a model that takes $S$ and $D$ and generate a probability of match between a Kud expense and a Toto expense in a 1:many match.<br>
To train that model you will first need to get a labeled dataset. <br>


### 2.1. Generation of Embeddings

The biggest problems in generating embeddings are: 

* **Language**. I'm using multi-lingual text and need to be able to generate comparable embeddings from text in different languages. I need a model with good cross-lingual transferability meaning that the learned sentence representations can capture semantic similarity and transfer knowledge across different languages. <br>
There are models like LASER or BERT Multilingual that can do that, but they are **very heavy**, hence it is not feasible to store them in a running container as part of an API.



In [46]:
import json

with open("payments.json", "r") as file: 
    toto_data = json.load(file)

In [47]:
kud_data = []

with open(kud_data_filename, "r") as file:
    for line in file:
        line = file.readline().replace('\n', '')
        kud_data.append(json.loads(line.replace('\'', '"')))


In [48]:
expenses_list = toto_data["expenses"]

# Only extract the list of expenses in the date range [10.2022, 12.2022]
selected_toto_data = []
for expense in expenses_list: 
    
    date = str(expense["date"])

    if int(date[:6]) >= 201801 and int(date[:6]) <= 202309: 
        selected_toto_data.append({
            "date": date[6:] + "." + date[4:6] + "." + date[:4], 
            "text": expense["description"],
            "amount": expense["amount"],
        })


In [56]:
def to_series(training_example): 

    data = [
        training_example["type"],
        training_example["kud"]["date"],
        training_example["kud"]["text"],
        training_example["kud"]["amount"], 
        training_example["toto"]["date"],
        training_example["toto"]["text"],
        training_example["toto"]["amount"], 
        training_example["label"]
    ]

    return data

def to_dataframe(training_examples): 

    data = []
    
    for ex in training_examples: 
        if ex["kud"]["text"] == "": 
            continue
        data.append(to_series(ex))

    return pd.DataFrame(data, columns=["type", "kud_date", "kud_text", "kud_amount", "toto_date", "toto_text", "toto_amount", "label"])

In [57]:
import copy 

# Now let's try a reconciliation
# List of reconciled expenses
reconciled = []

# List of potential matches
one_to_many = []

def month_of(date): 
    return date[3:5]

num_onetomany = 0

for kud_expense in kud_data: 

    amount_matches = []

    for toto_expense in selected_toto_data: 

        if toto_expense["amount"] == abs(kud_expense["amount"]) and month_of(toto_expense["date"]) == month_of(kud_expense["date"]): 
            amount_matches.append(toto_expense)

    if len(amount_matches) == 1: 
        reconciled.append({"kud": kud_expense, "toto": amount_matches[0]})
    elif len(amount_matches) > 1: 
        one_to_many.append({"kud": kud_expense, "toto": amount_matches})
        num_onetomany += len(amount_matches)


print(f"Number of matches: {len(reconciled)}")
print(f"Number of one-to-many: {num_onetomany}")

# Generate some negative examples
print(f"Will generate {len(reconciled)} negative examples")

# The logic for negative examples is to take a reconciled i-th example and take the kud expense and i+1-th toto expense. They will most likely mismatch
neg_examples = []
for i in range(len(reconciled)): 

    j = i+1

    if j == len(reconciled): 
        j = 0
    
    neg_example = {"kud": reconciled[i]["kud"], "toto": reconciled[j]["toto"]}

    neg_examples.append(neg_example)

training_examples = []

for item in one_to_many: 
    for toto_expense in item["toto"]: 
        training_example = {"type": "onetomany", "kud": item["kud"], "toto": toto_expense, "label": "?"}
        training_examples.append(training_example)

for item in reconciled: 
    training_example = {"type": "reconciled", "kud": item["kud"], "toto": item["toto"], "label": 1}
    training_examples.append(training_example)

for item in neg_examples: 
    training_example = {"type": "negative", "kud": item["kud"], "toto": item["toto"], "label": 0}
    training_examples.append(training_example)

df = to_dataframe(training_examples=training_examples)
df.to_csv("training.csv", index=False)


Number of matches: 754
Number of one-to-many: 401
Will generate 754 negative examples
